# Joins Types

In [1]:
# join experssion determine whether two rows should join.
# join types determine ,what should be in result set
# 1) inner join => keep rows with keys in either left and right datasets
# 2) outer join => keep rows with keys in either left or right datasets
# 3) left join  => keep rows with keys in the left dataset
# 4) right join => keep rows with keys in the right dataset
# 5) left semi join => keep rows in the left,and only left dataset , where the keys appears in the right dataset
# 6) left anti join => keep rows in the left,and only left dataset , where the keys do not appears in the right dataset
# 7) cross join  => match every row in the left datasets with every row in the right table 

In [2]:
# let's create our dataframes
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])])\
  .toDF("id", "name", "graduate_program", "spark_status")
graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")])\
  .toDF("id", "degree", "department", "school")
sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
  .toDF("id", "status")

In [3]:
# let's register dataframes to tables
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [6]:
joinexperssion = person["graduate_program"] == graduateProgram["id"]

In [11]:
# the default join is inner join
#person.join (graduateProgram ,joinexperssion ).show()
# , we can also explicitly passing  join type
jointype = "inner"
person.join (graduateProgram ,joinexperssion, jointype).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [9]:
spark.sql("select * from person join graduateProgram on  person.graduate_program = graduateProgram.id ").show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



## the above three sentences will repeat in every join statment with change the join types

In [12]:
jointype = "outer"
person.join (graduateProgram ,joinexperssion, jointype).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [13]:
jointype = "left_outer"
person.join (graduateProgram ,joinexperssion, jointype).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [14]:
jointype = "right_outer"
person.join (graduateProgram ,joinexperssion, jointype).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [15]:
jointype = "left_semi"
person.join (graduateProgram ,joinexperssion, jointype).show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



In [19]:
# create another datframe and join it with graduate program , to show this example 
gradprogram2 = graduateProgram.union(spark.createDataFrame([
    (0, "master","duplicate row" , " duplicated school")
]))
gradprogram2.join(person ,joinexperssion, jointype ).show()

+---+-------+--------------------+------------------+
| id| degree|          department|            school|
+---+-------+--------------------+------------------+
|  0|Masters|School of Informa...|       UC Berkeley|
|  1|  Ph.D.|                EECS|       UC Berkeley|
|  0| master|       duplicate row| duplicated school|
+---+-------+--------------------+------------------+



In [21]:
# left_anti : keep only values thes don't have corresponding values in other table. like ( not in ) in sql server
jointype = "left_anti"
graduateProgram.join (person ,joinexperssion, jointype).show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



In [26]:
# cross join 
# join every single row from the left table with all rows from the right join 
# explosion in num of rows , if we have dataframes with 1000 rows, the cross join will be 1000*1000 = 1 million rows 
jointype = "cross"
graduateProgram.join (person ,joinexperssion, jointype).show()
# but this will not produce cross join , because this is dangerous , so we can create it explict

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+



In [27]:
# we can call it explicitly
person.crossJoin(graduateProgram).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  2|Masters|                EECS|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  2|Masters|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  0|Masters|School of Informa...|UC 

### join in complex types like arrays

In [28]:
from pyspark.sql.functions import expr

In [34]:
person.withColumnRenamed("id", "personid")\
.join(sparkStatus, expr("array_contains(spark_status , id)")).show()

+--------+----------------+----------------+---------------+---+--------------+
|personid|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



In [37]:
spark.sql(" select * from (select id as persionid , name ,graduate_program ,spark_status from person )inner join sparkStatus on array_contains (spark_status , id) ").show()

+---------+----------------+----------------+---------------+---+--------------+
|persionid|            name|graduate_program|   spark_status| id|        status|
+---------+----------------+----------------+---------------+---+--------------+
|        0|   Bill Chambers|               0|          [100]|100|   Contributor|
|        1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|        1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|        1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|        2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|        2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+---------+----------------+----------------+---------------+---+--------------+



In [ ]:
# handling duplicate column names


# How Spark Perform Joins

In [ ]:
#there is two main concepts in joining , shuffle join  which  uses in very large tables and broadcast join  with uses in intermidate tables 
# in the small tables , we prefer to let spark determine how to join them 
# this is a good link https://towardsdatascience.com/the-art-of-joining-in-spark-dcbd33d693c

In [ ]:
#spark using Sort Merge Joins if we are joining two big tables, or Broadcast Joins if at least one of the datasets involved is small 
#enough to be stored in the memory of the single all executors. Note that there are other types of joins (e.g. Shuffle Hash Joins),
#but those mentioned earlier are the most common, in particular from Spark 2.3.